In [137]:
import pandas as pd

In [138]:
base_estoque = pd.read_excel("bases/bases_limpas/base_estoque_limpo.xlsx")
base_notas = pd.read_excel("bases/bases_limpas/base_notas_limpa.xlsx")

In [139]:
df = pd.merge(
    base_notas,
    base_estoque[["Código produto", "Código da categoria", "Categoria", "Código da Marca", "Marca", "Quantidade estoque"]],
    on='Código produto',
    how='inner' 
)

In [140]:
df["Margem bruta"] = (df["Valor unitário"] - df["Preço de custo"]).round(2)
df["Margem %"] = (df["Margem bruta"] / df["Valor unitário"]).round(2)
df["Markup"] = (df["Valor unitário"] - df["Preço de custo"]).round(2)

In [141]:
df["Data da venda"] = pd.to_datetime(df["Data da venda"], format='%d/%m/%Y')
df["Quantidade do produto"] = df["Quantidade do produto"].astype(int)
df["Código da categoria"] = df["Código da categoria"].astype(int)
df["Código da Marca"] = df["Código da Marca"].astype(int)
df["Quantidade estoque"] = df["Quantidade estoque"].astype(int)


In [142]:
# filtrar linhas onde o preço de custo é nulo
itens_sem_preco_custo = df[df["Preço de custo"].isna()]
# extrair os números das notas com itens problemáticos
notas_afetadas = itens_sem_preco_custo["Numero nota fiscal"].unique()
# contar quantas notas são impactadas
total_notas_afetadas = len(notas_afetadas)
total_notas_unicas = df["Numero nota fiscal"].nunique()
percentual_afetado = (total_notas_afetadas / total_notas_unicas) * 100


In [143]:
# 1. Lista das notas fiscais que têm pelo menos um item com "Preço de custo" nulo
notas_com_problema = df[df["Preço de custo"].isna()]["Numero nota fiscal"].unique()

# 2. Criar um novo DataFrame excluindo essas notas
df_completo_limpo = df[~df["Numero nota fiscal"].isin(notas_com_problema)]


In [144]:
# verificar_base_mesclada(df_completo_limpo)

In [145]:
# def verificar_valores_zerados(df):
#     """
#     Verifica valores zerados em colunas numéricas da base mesclada.
    
#     Parâmetros:
#         df (pd.DataFrame): DataFrame a ser analisado
    
#     Retorna:
#         DataFrame com contagem de zeros por coluna numérica
#     """
#     # Seleciona apenas colunas numéricas
#     colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
    
#     # Conta valores zerados em cada coluna numérica
#     zeros_por_coluna = []
#     for coluna in colunas_numericas:
#         qtd_zeros = (df[coluna] == 0).sum()
#         zeros_por_coluna.append({'Coluna': coluna, 'Valores Zerados': qtd_zeros})
    
#     # Cria e retorna DataFrame com resultados
#     resultado = pd.DataFrame(zeros_por_coluna)
#     resultado['Percentual'] = (resultado['Valores Zerados'] / len(df) * 100).round(2)
    
#     # Ordena por quantidade de zeros (decrescente)
#     return resultado.sort_values('Valores Zerados', ascending=False)

# # Como usar:
# # resultado_zeros = verificar_valores_zerados(df_mesclado)
# # display(resultado_zeros)

In [146]:
# resultado_zeros = verificar_valores_zerados(df_completo_limpo)
# display(resultado_zeros)

In [147]:
# 1. Identificar as notas fiscais com Valor unitário zerado
notas_com_valor_zerado = df[df['Valor unitário'] == 0]['Numero nota fiscal'].unique()
total_notas_afetadas = len(notas_com_valor_zerado)

print(f"🔍 Notas fiscais com Valor unitário zerado: {total_notas_afetadas}")

# 2. Criar uma versão limpa do DataFrame excluindo essas notas
df_limpo = df[~df['Numero nota fiscal'].isin(notas_com_valor_zerado)].copy()

# 3. Recalcular o Valor total produto e Valor da nota
df_limpo['Valor total produto'] = df_limpo['Quantidade do produto'] * df_limpo['Valor unitário']
df_limpo['Valor da nota'] = df_limpo.groupby('Numero nota fiscal')['Valor total produto'].transform('sum')

# 5. (Opcional) Análise detalhada das notas removidas
if total_notas_afetadas > 0:
    print("\n📊 Detalhes das notas removidas:")
    notas_removidas = df[df['Numero nota fiscal'].isin(notas_com_valor_zerado)]
    display(notas_removidas.groupby(['Numero nota fiscal', 'Descrição do produto'])[['Quantidade do produto', 'Valor unitário']].sum())

🔍 Notas fiscais com Valor unitário zerado: 85

📊 Detalhes das notas removidas:


Quantidade do produto  \
Numero nota fiscal Descrição do produto                                         
37033              ACUCAR CRISTAL 1KG SAMUKA                                0   
                   AGUA SANITARIA 5L TA LIMPEZA                             0   
                   AROMAT LAVANDA FLORAL 400ML LEV&UZE                      0   
                   CAFE TRADICIONAL VACUO 250G UNIAO                        0   
                   COLA INSTANTANEA 20G TEKBOND 725                         0   
...                                                                       ...   
564625             SACO 45X59 SORTIDO FEM C/25 CROMUS                       1   
                   SACO 45X59 SORTIDO MASC C/25 CROMUS                      1   
                   SACO 50X70 ROMANTICO C/25 CROMUS                         1   
565290             SUCATA DE PAPELAO                                        0   
565870             DISPLAY BALCAO SPIRO CIS                                 1   

                                                        Valor unitário  
Numero nota fiscal Descrição do produto                                 
37033              ACUCAR CRISTAL 1KG SAMUKA                       0.0  
                   AGUA SANITARIA 5L TA LIMPEZA                    0.0  
                   AROMAT LAVANDA FLORAL 400ML LEV&UZE             0.0  
                   CAFE TRADICIONAL VACUO 250G UNIAO               0.0  
                   COLA INSTANTANEA 20G TEKBOND 725                0.0  
...                                                                ...  
564625             SACO 45X59 SORTIDO FEM C/25 CROMUS              0.0  
                   SACO 45X59 SORTIDO MASC C/25 CROMUS             0.0  
                   SACO 50X70 ROMANTICO C/25 CROMUS                0.0  
565290             SUCATA DE PAPELAO                               0.0  
565870             DISPLAY BALCAO SPIRO CIS                        0.0  

[637 rows x 2 columns]

In [148]:
# resultados_negativos = analisar_valores_negativos(df_completo_limpo)

In [149]:
def excluir_notas_com_negativos(df, colunas_verificar=None):
    """
    Remove todas as notas fiscais que contêm valores negativos nas colunas especificadas.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados mesclados
        colunas_verificar (list): Lista de colunas para verificar valores negativos
                                 Se None, verifica todas as colunas numéricas
    
    Retorno:
        pd.DataFrame: DataFrame limpo sem as notas problemáticas
        pd.DataFrame: DataFrame com as notas removidas (para auditoria)
    """
    # 1. Definir colunas para verificação
    if colunas_verificar is None:
        colunas_numericas = df.select_dtypes(include=['int64', 'float64']).columns
        colunas_verificar = [col for col in colunas_numericas if col not in ['Numero nota fiscal', 'Código produto']]
    
    # 2. Identificar registros com valores negativos
    mascara_negativos = pd.Series(False, index=df.index)
    for coluna in colunas_verificar:
        mascara_negativos |= (df[coluna] < 0)
    
    # 3. Identificar notas fiscais problemáticas
    notas_com_negativos = df.loc[mascara_negativos, 'Numero nota fiscal'].unique()
    print(f"🔍 Encontradas {len(notas_com_negativos)} notas fiscais com valores negativos")
    
    # 4. Criar DataFrames para resultado e auditoria
    df_limpo = df[~df['Numero nota fiscal'].isin(notas_com_negativos)].copy()
    df_removido = df[df['Numero nota fiscal'].isin(notas_com_negativos)].copy()
    
    # 5. Recalcular valores totais na base limpa
    df_limpo['Valor total produto'] = df_limpo['Quantidade do produto'] * df_limpo['Valor unitário']
    df_limpo['Valor da nota'] = df_limpo.groupby('Numero nota fiscal')['Valor total produto'].transform('sum')
    
    # 6. Relatório de impacto
    print("\n✅ Resultado da limpeza:")
    print(f"- Total original de notas: {df['Numero nota fiscal'].nunique()}")
    print(f"- Notas removidas: {len(notas_com_negativos)}")
    print(f"- Notas restantes: {df_limpo['Numero nota fiscal'].nunique()}")
    print(f"- Registros removidos: {len(df_removido)}")
    print(f"- Percentual removido: {len(df_removido)/len(df):.2%}")
    
    return df_limpo, df_removido

# Como usar:
# df_limpo, df_removido = excluir_notas_com_negativos(df_mesclado, colunas_verificar=['Quantidade do produto', 'Valor unitário', 'Preço de custo'])

# Opção para verificar todas colunas numéricas:
# df_limpo, df_removido = excluir_notas_com_negativos(df_mesclado)

In [150]:
df_limpo, df_removido = excluir_notas_com_negativos(df_completo_limpo)

🔍 Encontradas 276 notas fiscais com valores negativos

✅ Resultado da limpeza:
- Total original de notas: 24285
- Notas removidas: 276
- Notas restantes: 24009
- Registros removidos: 3134
- Percentual removido: 1.77%


In [151]:
def excluir_notas_com_custo_maior_que_venda(df):
    """
    Remove todas as notas fiscais que contêm itens onde:
    Preço de custo > Valor unitário
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados mesclados
        
    Retorna:
        pd.DataFrame: Base limpa sem as notas problemáticas
        pd.DataFrame: Notas removidas (para auditoria)
    """
    # 1. Identificar itens problemáticos
    mascara_problema = df['Preço de custo'] > df['Valor unitário']
    itens_problematicos = df[mascara_problema]
    
    # 2. Pegar números das notas fiscais problemáticas
    notas_problematicas = itens_problematicos['Numero nota fiscal'].unique()
    total_notas_problema = len(notas_problematicas)
    
    print(f"🔍 Encontradas {total_notas_problema} notas com itens onde preço de custo > valor unitário")
    print(f"📦 Itens problemáticos identificados: {len(itens_problematicos)}")
    
    # 3. Criar versão limpa (excluindo as notas problemáticas)
    df_limpo = df[~df['Numero nota fiscal'].isin(notas_problematicas)].copy()
    
    # 4. Criar dataframe com as notas removidas (para auditoria)
    df_removido = df[df['Numero nota fiscal'].isin(notas_problematicas)].copy()
    
    # 5. Recalcular valores totais na base limpa
    df_limpo['Valor total produto'] = df_limpo['Quantidade do produto'] * df_limpo['Valor unitário']
    df_limpo['Valor da nota'] = df_limpo.groupby('Numero nota fiscal')['Valor total produto'].transform('sum')
    
    # 6. Relatório de impacto
    print("\n✅ RESULTADO DA LIMPEZA:")
    print(f"- Notas originais: {df['Numero nota fiscal'].nunique()}")
    print(f"- Notas removidas: {total_notas_problema}")
    print(f"- Notas restantes: {df_limpo['Numero nota fiscal'].nunique()}")
    print(f"- Registros removidos: {len(df_removido)}")
    print(f"- Percentual removido: {len(df_removido)/len(df):.2%}")
    
    return df_limpo, df_removido

# Como usar:
# df_limpo, df_removido = excluir_notas_com_custo_maior_que_venda(df_mesclado)

# Para exportar os resultados:
# df_limpo.to_excel('base_limpa.xlsx', index=False)
# df_removido.to_excel('notas_removidas_auditoria.xlsx', index=False)

In [152]:
df_limpo, df_removido = excluir_notas_com_custo_maior_que_venda(df_completo_limpo)

🔍 Encontradas 275 notas com itens onde preço de custo > valor unitário
📦 Itens problemáticos identificados: 446

✅ RESULTADO DA LIMPEZA:
- Notas originais: 24285
- Notas removidas: 275
- Notas restantes: 24010
- Registros removidos: 3128
- Percentual removido: 1.77%


In [153]:
def excluir_notas_problematicas(df):
    """
    Exclui notas fiscais afetadas por:
    - Preço de custo zerado
    - Valor da nota zerado
    - Quantidade do produto negativa
    - Valor total produto negativo
    """
    # Identificar notas problemáticas
    notas_problema = set()
    
    # 1. Notas com Preço de custo zerado
    notas_problema.update(df[df['Preço de custo'] == 0]['Numero nota fiscal'].unique())
    
    # 2. Notas com Valor da nota zerado
    notas_problema.update(df[df['Valor da nota'] == 0]['Numero nota fiscal'].unique())
    
    # 3. Notas com Quantidade negativa
    notas_problema.update(df[df['Quantidade do produto'] < 0]['Numero nota fiscal'].unique())
    
    # 4. Notas com Valor total negativo
    notas_problema.update(df[df['Valor total produto'] < 0]['Numero nota fiscal'].unique())
    
    # Criar DataFrame limpo
    df_limpo = df[~df['Numero nota fiscal'].isin(notas_problema)].copy()
    
    # Relatório
    print(f"Notas originais: {df['Numero nota fiscal'].nunique()}")
    print(f"Notas removidas: {len(notas_problema)}")
    print(f"Notas restantes: {df_limpo['Numero nota fiscal'].nunique()}")
    print(f"Registros removidos: {len(df) - len(df_limpo)}")
    
    return df_limpo



In [154]:
df_limpo = excluir_notas_problematicas(df_limpo)

Notas originais: 24010
Notas removidas: 2
Notas restantes: 24008
Registros removidos: 16


In [155]:
df_limpo.to_csv("bases/bases_limpas/base_para_modelo_final.csv", index=False)